In [55]:
import numpy as np
#import modin.pandas as pd
import pandas as pd
from ppca import PPCA
import six

import warnings

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, Birch, FeatureAgglomeration, SpectralClustering, SpectralCoclustering, SpectralBiclustering, DBSCAN, AffinityPropagation, MeanShift
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.decomposition import PCA, FastICA, IncrementalPCA, KernelPCA, FactorAnalysis, TruncatedSVD, SparsePCA, MiniBatchSparsePCA, DictionaryLearning, MiniBatchDictionaryLearning
from sklearn.manifold import TSNE, MDS, SpectralEmbedding, LocallyLinearEmbedding, Isomap
from umap import UMAP

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

#import plotly_express as px

In [56]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow.compat.v1 as tf
#import tensorflow as tf
import tensorflow_hub as hub

tf.disable_v2_behavior()
tf.disable_eager_execution()

In [57]:
from srm import SRM
from procrustes import procrustes

#working from align.py:
# - include SRM, hyperalignment, and procrustes as options (for procrustes, align everything to the first observation)
#   - modify these functions to return both the transformed data and also a function that will map a new set of observations into the common space
# - format the data to ensure it's a list of dataframes
# - carve out hyperalignment as a separate function (like SRM)
# - use the indices to create a common set of dataframes, all with indices from the full set (union) of indices across all dataframes in the list, and all with the same number of columns (zero-padding to match the dataframe with the most columns).  compute the transformations
#   using just the common indices (intersection).  But the transform function should work with arbitrary data, including from non-matching indices.
#   The full align function should return a list of dataframes of the same size/shape as the inputted list, and where each dataframe has the same number of rows (at the same indices)
#   as the corresponding dataframes in the original list, but every dataframe in the aligned list has the same number of columns.

#consider: returnning a new sub-class of dataframe that includes the inverse transformation functions, allowing the data to be transformed back into the original data spaces
#in turn, this would return a new TransformableDataFrame that could map the (original) data back into the transformed space.

In [58]:
from configparser import ConfigParser
#from collections import defaultdict

In [67]:
defaults = ConfigParser()
defaults.read('defaults.ini');

In [60]:
init_notebook_mode(connected=True)

In [61]:
reduce_models = ['DictionaryLearning', 'FactorAnalysis', 'FastICA', 'IncrementalPCA', 'KernelPCA', 'LatentDirichletAllocation', 'MiniBatchDictionaryLearning',
 'MiniBatchSparsePCA', 'NMF', 'PCA', 'SparseCoder', 'SparsePCA', 'TruncatedSVD', 'UMAP', 'TSNE', 'MDS', 'SpectralEmbedding', 'LocallyLinearEmbedding', 'Isomap']

cluster_models = ['AffinityPropagation', 'AgglomerativeClustering', 'Birch', 'DBSCAN', 'FeatureAgglomeration', 'KMeans', 'MeanShift', 'MiniBatchKMeans', 'SpectralBiclustering', 'SpectralClustering', 'SpectralCoclustering', 'DBSCAN', 'AffinityPropagation', 'MeanShift']
mixture_models = ['GaussianMixture', 'BayesianGaussianMixture', 'LatentDirichletAllocation', 'NMF']
interpolation_models = ['linear', 'time', 'index', 'pad', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial']
text_models = ['universal_sentence_encoder', 'LatentDirichletAllocation', 'NMF']
corpora = ['wiki', 'nips', 'sotus']


### API ideas

1. [x] Write a single function to format data into a list of pandas dataframes:
    - [x] If text, should vectorize text according to default (or given) models.  Allow user to (optionally) pass in row/column labels for the text data.  Each string gets one row; a list of strings yields a len(list) by n-features dataframe
    - [x] If numpy arrays, set indices to row count and columns to column count
    - [x] If pandas dataframe, keep as is but turn into a list
    - [x] If only a single matrix is given, still return a list
    - [x] Dimensions may be mismatched
    - [x] Missing data is left unchanged
2. [x] Write a function to take a list of dataframes and use PPCA to fill in missing values.  Also use interpolation to fill in missing rows
3. Write a reduce function that can work in batch mode (reduce a single dataframe or a list of dataframes)
4. Write a cluster function that can work in batch mode (return cluster labels or mixture proportions for a single dataframe or a list of dataframes)
5. Write an align function that aligns all dataframes in a list into a common space
6. Write a normalize function that z-scores the data
7. Write a plot function that takes a list of dataframes and plots them
    - Need a mat2colors function that colorizes data points according to group labels, cluster assignments, mixture proportions, or user-specified matrices.  Take in a colormap as an argument
    - Need support for multicolored lines in addition to per-observation marker colors.  For 3d plots, could use streamtubes...but for 2d plots this won't work, so maybe a segment solution would be appropriate.
    - Potentially allow for different line/marker colors
8. Write a hyper-function that takes in a "messy" dataset, applies formatting, PPCA, reduce, alignment, and clustering (as specified) and returns a formatted/processed dataset.
9. Add hyper-function call to every helper function, so that all analyses can be applied from any call to any other function

In [62]:
def is_dataframe(x):
    return type(x).__module__ in ['pandas.core.frame', 'modin.pandas.dataframe']

def is_multiindex_dataframe(x):
    return is_dataframe(x) and (type(x.index).__module__ == 'pandas.core.indexes.multi')

def is_array(x):
    return type(x).__module__ == 'numpy'

def is_empty(x):
    return (x is None) or (len(x) == 0)

def is_text(x):
    if type(x) == list:
        return np.all([is_text(t) for t in x])
    return type(x) in six.string_types

In [73]:
def text_vectorizer(text, model='universal_sentence_encoder', **kwargs):
    def universal_sentence_encoder(text, **kwargs):        
        if 'USE_corpus' in kwargs.keys():
            corpus = kwargs['USE_corpus']
        else:
            corpus = defaults['text']['USE_corpus']
            #if not os.path.exists(corpus):
            #    os.system(f"'https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed' | tar -zxvC {defaults['text']['USE_corpus']}")
        
        assert os.path.exists(corpus), f'Corpus not found: {corpus}'

        encoder = hub.Module(corpus)

        tf.logging.set_verbosity(tf.logging.ERROR)
        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])        
            return session.run(encoder(text))
    
    def sklearn_vectorizer(text, model, **kwargs):
        if 'corpus' in kwargs.keys():
            corpus = kwargs['corpus']
        else:
            corpus = defaults['text']['corpus']
        
        assert (corpus in corpora) or ((type(corpus) == list) and is_text(corpus)) or os.path.exists(corpus), f'Cannot use corpus: {corpus}'
        
        if corpus in corpora:
            pass #FINISH THIS FUNCTION...
        
    
    assert (model in text_models) or (callable(model)), f'Unsupported model: {model}'
    if not (type(text) == list):
        text = [text]
    
    if callable(model):
        return model(text, **kwargs)
    elif model == 'universal_sentence_encoder':
        return universal_sentence_encoder(text, **kwargs)
    else:
        return sklearn_vectorizer(text, model, **kwargs)
    

In [76]:
def panda_stack(data, force=True, names=None, keys=None, verify_integrity=False, sort=False, copy=True, ignore_index=False, levels=None, **kwargs):
    '''
    Take a list of DataFrames with the same number of columns and (optionally)
    a list of names (of the same length as the original list; default:
    range(len(x))).  Return a single MultiIndex DataFrame where the original
    DataFrames are stacked vertically, with the data names as their level 1
    indices and their original indices as their level 2 indices.
    
    INPUTS
    data: data in any format (text, numpy arrays, pandas dataframes, or a mixed list (or nested lists) of those types)
    text_vectorizer: function that takes a string (or list of strings) and returns a numpy array or dataframe.  If
    force is False, must pass in a list of DataFrames.
    
    force: if True, use format_data to coerce everything into a list of pandas dataframes.
    
    text_vectorizer: function for turning text data into DataFrames, used if force is True
    
    Also takes all keyword arguments from pandas.concat except axis, join, join_axes
    
    All other keywork arguments (if any) are passed to text_vectorizer
    
    OUTPUTS
    a single MultiIndex DataFrame
    '''
    
    if force:
        data = format_data(data, **kwargs)
    
    assert type(data) == list, 'Must either pass a list of DataFrames or set force to True'
    assert np.all([is_dataframe(d) for d in data]), 'Must either pass a list of DataFrames or set force to True'
    assert len(np.unique([d.shape[1] for d in data])) == 1, 'All DataFrames must have the same number of columns'    
    template = data[0].columns.values
    for i, d1 in enumerate(data):
        template = d1.columns.values
        for d2 in data[(i+1):]:
            assert np.all([(c in template) for c in d2.columns.values]), 'All DataFrames must have the same columns'
    
    if keys is None:
        keys = np.arange(len(data), dtype=np.int)
    
    assert is_array(keys) or (type(keys) == list), f'keys must be None or a list or array of length len(data)'
    assert len(keys) == len(data), f'keys must be None or a list or array of length len(data)'
    
    return pd.concat(data, axis=0, join='outer', join_axes=None, names=names, keys=keys, 
                     verify_integrity=verify_integrity, sort=sort, copy=copy, ignore_index=ignore_index, levels=levels)

In [16]:
def panda_unstack(x):
    if not is_multiindex_dataframe(x):
        if is_dataframe(x):
            return x
        else:
            raise Exception(f'Unsupported datatype: {type(x)}')
    
    names = list(x.index.names)
    grouper = 'ID'
    if not (grouper in names):
        names[0] = grouper
    elif not (names[0] == grouper):
        for i in np.arange(len(names)): #trying n things other than 'ID'; at least one of them must be outside of the n-1 remaining names
            next_grouper = f'{grouper}{i}'
            if not (next_grouper in names):
                names[0] = next_grouper
                grouper = next_grouper
                break
    assert names[0] == grouper, 'Unstacking error'
    
    x.index.rename(names, inplace=True)    
    return [d[1].set_index(d[1].index.get_level_values(1)) for d in list(x.groupby(grouper))]

In [42]:
def format_data(x, **kwargs):
    '''
    INPUTS
    x: data in any format (text, numpy arrays, pandas dataframes, or a mixed list (or nested lists) of those types)
    text_vectorizer: function that takes a string (or list of strings) and returns a numpy array or dataframe
    text_kwargs: dictionary of keywork arguments to pass to text_vectorizer
    
    OUTPUTS
    a list of pandas dataframes
    '''
    
    def to_dataframe(y):
        if is_dataframe(y):
            return y
        elif is_array(y):
            assert np.ndim(y) == 2, 'all data matrices must be 2-dimensional'
            idx = np.arange(y.shape[0])
            cols = np.arange(y.shape[1])
            return pd.DataFrame(data=y, index=idx, columns=cols)
        elif type(y) == list:
            return [to_dataframe(i) for i in y]
        elif is_text(y):
            if 'text_model' in kwargs.keys():
                text_model = kwargs['text_model']
            else:
                text_model = defaults['text']['model']
                
            if 'text_kwargs' in kwargs.keys():
                text_kwargs = kwargs['text_kwargs']
            else:
                text_kwargs = {}
            return to_dataframe(text_vectorizer(y, model=text_model, **text_kwargs))
        else:
            raise NotImplementedError(f'Unsupported data type: {type(y)}')
    
    if type(x) == list:
        return [to_dataframe(i) for i in x]
    else:
        return [to_dataframe(x)]

In [18]:
def fill_missing(x, apply_ppca=True, interpolation='linear', **kwargs):
    if type(x) == list:
        return [fill_missing(i, apply_ppca=apply_ppca, interpolation=interpolation, **kwargs) for i in x]
    elif is_array(x):
        return fill_missing(pd.DataFrame(x), apply_ppca=apply_ppca, interpolation=interpolation, **kwargs).values
    elif not is_dataframe(x):
        raise Exception('Unsupported datatype: f{type(x)}')
    
    if 'method' in interpolate_kwargs.keys():
        warnings.warn(f"Overloaded keyword argument; ignoring interpolation parameter ({interpolation}) in favor of keyword-specified method: {interpolate_kwargs['method']}")
    else:
        interpolate_kwargs['method'] = interpolation
    
    assert interpolate_kwargs['method'] in interpolation_models, f"Unsupported interpolation type: '{interpolate_kwargs['method']}'"
    
    if apply_ppca:
        covariance_model = PPCA()
        covariance_model.fit(x.values)
        x.values = covariance_model.transform()
    
    if not ('inplace' in kwargs.keys()):
        kwargs['inplace'] = False
    
    if not ('axis' in kwargs.keys()):
        kwargs['axis'] = 0
        
    if 'method' in kwargs.keys():
        warnings.warn(f"Overloaded keyword argument; ignoring interpolation parameter ({interpolation}) in favor of keyword-specified method: {kwargs['method']}")
    else:
        kwargs['method'] = interpolation
    
    return x.interpolate(**kwargs)

In [19]:
#TODO:
#write a function that aligns the indices and columns of a list of dataframes

In [20]:
# def pandify(idx, cols, vals, force=False):
#     if (len(idx) == 0) or (idx is None):
#         idx = np.arange(vals.shape[0])
#     if (len(cols) == 0) or (cols is None):
#         cols = np.arange(ndims)
    
#     if is_dataframe(vals):
#         vals = vals.copy()
#         vals.index = idx
#         vals.columns = cols
#         return vals
#     else:
#         return 
    

# def depandify(x):
#     if is_dataframe(data):
#         idx = data.index
#         cols = np.arange(ndims)
#         vals = data.values
#     elif is_array(data):
#         idx = []
#         cols = []
#         vals = data
#     else:
#         raise Exception(f'Unsupported data type: {type(data)}')
#     return idx, cols, vals

    

In [21]:
#re-write reducer and batch_reducer to use new functions...

In [80]:
def reduce(data, algorithm='IncrementalPCA', stack=False, ndims=3, fillna=False, interpolation_kwargs={}, force_pandas=False, **kwargs):
    '''
    ARGUMENTS:
    data: data to reduce (numpy array or compatable, or a pandas
          dataframe or compatable).  Formatted as a 2d matrix whose
          rows are observations and whose columns are feature
          dimensions.
    
    force_pandas: if True, force the result to be a pandas dataframe;
          if False (default), return the inputs in the same format
          as they were provided (unless stack=True, in which case
          force_pandas is reset to True).
    
    algorithm: one of: 'DictionaryLearning', 'FactorAnalysis',
          'FastICA', 'IncrementalPCA', 'KernelPCA',
          'LatentDirichletAllocation', 'MiniBatchDictionaryLearning',
          'MiniBatchSparsePCA', 'NMF', 'PCA', 'SparseCoder',
          'SparsePCA', 'TruncatedSVD', or 'UMAP'.  Calls the
          relevant scikit-learn or UMAP function.  Can also
          pass a function directly.
    
    ndims: the number of dimensions (columns) in the result (default: 3).
          if ndims > data.shape[1], the right-most columns will be
          zero-padded.
    
    fillna: if True, use PPCA and interpolation to fill in nan-valued entries (default: False)
    
    all additional keyword arguments are passed to the reduce algorithm
    
    RETURNS:
    numpy array or pandas dataframe with number-of-observations rows and
    ndims columns
    '''
    
    if type(algorithm) == str:
        assert algorithm in reduce_models, f'Unknown algorithm: {algorithm}' #security check to prevent executing arbitrary code
        algorithm = eval(algorithm)
    else:
        if ('sklearn.decomposition' in algorithm.__module__) or ('umap.umap_' in algorithm.__module__):
            pass #algorithm already given as a function
    
    assert ndims >= 0, f'Number of dimensions must be non-negative (given: {ndims})'
    
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    
    data = format_data(data, **kwargs)
    
    if fillna:
        data = fill_missing(data, **interpolation_kwargs)
    
    stacked_data = panda_stack(data)
    vals = stacked_data.values
    
    if vals.shape[1] > ndims:
        model = algorithm(n_components=ndims, **kwargs)
        reduced_vals = model.fit_transform(vals)
    elif vals.shape[1] < ndims:
        reduced_vals = np.concatenate([vals, np.zeros([vals.shape[0], ndims-vals.shape[1]])], axis=1)
    else:
        reduced_vals = vals
    
    reduced_data = pd.DataFrame(data=reduced_vals, index=stacked_data.index, columns=np.arange(ndims))
    if stack:
        return reduced_data
    else:
        return panda_unstack(reduced_data)

In [78]:
# def reducer(data=None, algorithm='IncrementalPCA', ndims=3, fill_missing=False, force_pandas=False, **kwargs):
#     '''
#     ARGUMENTS:
#     data: data to reduce (numpy array or compatable, or a pandas
#           dataframe or compatable).  Formatted as a 2d matrix whose
#           rows are observations and whose columns are feature
#           dimensions.
    
#     force_pandas: if True, force the result to be a pandas dataframe;
#           if False (default), return the inputs in the same format
#           as they were provided (unless stack=True, in which case
#           force_pandas is reset to True).
    
#     algorithm: one of: 'DictionaryLearning', 'FactorAnalysis',
#           'FastICA', 'IncrementalPCA', 'KernelPCA',
#           'LatentDirichletAllocation', 'MiniBatchDictionaryLearning',
#           'MiniBatchSparsePCA', 'NMF', 'PCA', 'SparseCoder',
#           'SparsePCA', 'TruncatedSVD', or 'UMAP'.  Calls the
#           relevant scikit-learn or UMAP function.  Can also
#           pass a function directly.
    
#     ndims: the number of dimensions (columns) in the result (default: 3).
#           if ndims > data.shape[1], the right-most columns will be
#           zero-padded.
    
#     fill_missing: if True, use PPCA to fill in nan-valued entries
    
#     all additional keyword arguments are passed to the reduce algorithm
    
#     RETURNS:
#     numpy array or pandas dataframe with number-of-observations rows and
#     ndims columns
#     '''
#     if type(algorithm) == str:
#         assert algorithm in reduce_models, f'Unknown algorithm: {algorithm}' #security check to prevent executing arbitrary code
#         algorithm = eval(algorithm)
#     else:
#         if ('sklearn.decomposition' in algorithm.__module__) or ('umap.umap_' in algorithm.__module__):
#             pass #algorithm already given as a function
#     assert ndims >= 0, f'Number of dimensions must be non-negative (given: {ndims})'
    
#     import warnings
#     warnings.filterwarnings('ignore', category=RuntimeWarning)
    
#     if fill_missing:
#         raise(NotImplementedError('call PPCA and/or use interpolation to fill in missing values'))
    
#     if data.shape == ndims:
#         return data
    
#     if is_dataframe(data):
#         idx = data.index
#         cols = np.arange(ndims)
#         vals = data.values
#     elif is_array(data):
#         idx = []
#         cols = []
#         vals = data
#     else:
#         raise Exception(f'Unsupported data type: {type(data)}')
    
#     if force_pandas:
#         if len(idx) == 0:
#             idx = np.arange(vals.shape[0])
#         if len(cols) == 0:
#             cols = np.arange(ndims)
    
#     if vals.shape[1] > ndims:
#         model = algorithm(n_components=ndims, **kwargs)
#         reduced_vals = model.fit_transform(vals)
#     elif vals.shape[1] < ndims:
#         reduced_vals = np.concatenate([vals, np.zeros([vals.shape[0], ndims-vals.shape[1]])], axis=1)
#     else:
#         reduced_vals = vals
    
#     if is_dataframe(data) or force_pandas:
#         return pd.DataFrame(data=reduced_vals, index=idx, columns=cols)
#     else:
#         return reduced_vals

In [79]:
# def batch_reducer(data=None, stack=False, names=None, **kwargs):
#     '''
#     ARGUMENTS:
#     data: a numpy array (or compatable), pandas dataframe 
#           (or compatable), or a list of arrays and/or dataframes.
#           If multiple matrices are passed (e.g. in a list), they
#           must all have the same number of columns.
    
#     stack: should the answer be returned as a list (stack=False,
#           default) or a single multiindexed pandas dataframe
#           (stack=True)?
    
#     names: a list or array of strings, of the same length as data
#           (or, if data isn't a list, just a single string),
#           describing each dataset (e.g. for use in a legend)
    
#     force_pandas: if True, force the result to be a pandas dataframe;
#           if False (default), return the inputs in the same format
#           as they were provided (unless stack=True, in which case
#           force_pandas is reset to True).
    
#     algorithm: one of: 'DictionaryLearning', 'FactorAnalysis',
#           'FastICA', 'IncrementalPCA', 'KernelPCA',
#           'LatentDirichletAllocation', 'MiniBatchDictionaryLearning',
#           'MiniBatchSparsePCA', 'NMF', 'PCA', 'SparseCoder',
#           'SparsePCA', 'TruncatedSVD', or 'UMAP'.  Calls the
#           relevant scikit-learn or UMAP function.  Can also
#           pass a function directly.
    
#     all additional keyword arguments are passed to the reduce algorithm
    
#     RETURNS:
    
#     One or more reduced-dimensional matrices (each of shape
#     number-of-observations by ndims), in the same format as the inputted
#     data (or as pandas dataframes if force_pandas is True).  Or, if 
#     stack=True, return a single multilevel dataframe.    
#     '''
#     if not type(data) == list:
#         reduced = batch_reducer(data=[data], stack=stack, names=names, **kwargs)
#         if stack:
#             return reduced
#         else:
#             return reduced[0]
    
#     assert len(np.unique([x.shape[1] for x in data])) == 1, 'All datasets must have the same number of columns'
#     assert np.all([np.ndim(x) == 2 for x in data]), 'All datasets must be 2d arrays, matrices, or DataFrames'
    
#     rows = list(np.cumsum([x.shape[0] for x in data]))
#     rows.insert(0, 0)
    
#     #there's probably a cleaner way to do this...
#     if (('force_pandas' in kwargs.keys()) and kwargs['force_pandas']) or stack:
#         force_pandas = True
#         for i, d in enumerate(data):
#             if not is_dataframe(d):
#                 data[i] = pd.DataFrame(data=d, index=np.arange(d.shape[0]), columns=np.arange(d.shape[1]))
#     else:
#         force_pandas = False
    
#     modules = [type(x).__module__ for x in data]
#     idx = []
#     for i, d in enumerate(data):
#         if is_dataframe(d):
#             idx.append(d.index)
#             data[i] = data[i].values
#         else:
#             idx.append([])
    
#     x_combined = reducer(np.concatenate(data, axis=0), **kwargs)
#     if names is None:
#         names = np.arange(len(data), dtype=np.int)
    
#     if stack: #put everything into a multilevel index
#         groups = np.zeros(x_combined.shape[0])
#         for i, j in enumerate(rows[:-1]):
#             groups[j:rows[i+1]] = names[i]
#         idx = pd.MultiIndex.from_arrays([groups, [j for i in idx for j in i]], names=('ID', 'Observation'))
#         return pd.DataFrame(data=x_combined.values, index=idx, columns=np.arange(x_combined.shape[1]))
    
#     x_split = []
#     for i, j in enumerate(rows[:-1]):
#         if is_dataframe(x_combined):
#             x = x_combined.iloc[j:rows[i+1], :].values
#         else:
#             x = x_combined[j:rows[i+1], :]
        
#         if is_dataframe(data[i]):
#             x_split.append(pd.DataFrame(data=x, index=idx[i], columns=np.arange(x.shape[1])))
#         else:
#             x_split.append(x)
#     return x_split

In [24]:
def clusterer(data=None, algorithm='IncrementalPCA', ndims=3, fill_missing=False, force_pandas=False, **kwargs):
    raise NotImplementedError('write this...')
    return None

In [81]:
def plotter(data, style='line', reduce_kwargs={}, **kwargs):
        
    def combo_merge(a, b):
        combos = []
        for i in a:
            for j in b:
                if (len(i) <= 2):
                    if (len(j) <= 2):
                        combos.append(i+j)
                elif len(j) > 1:
                    combos.append(i + '+' + j)
        return combos
    
    marker_styles = ['.', 'o', 'scatter', 'marker', 'markers', 'bigmarker', 'bigmarkers']
    line_styles = ['-', '--', ':', '-:', 'line', 'lines']
    combo_styles = combo_merge(marker_styles, line_styles) + combo_merge(line_styles, marker_styles)
    big_markers = ['o', 'big']
    dash_styles = {'--': 'dash', '-:': 'dashdot', ':': 'dot'}
    
    def substr_list(style, x):
        '''
        style: a style string
        x: a list of substrings
        
        return: true if any of the strings in x is a substring of s, and false othewise    
        '''
        inds = np.array([s in style for s in x])
        if np.any(inds):
            return x[np.where(inds)[0][0]]
        else:
            return ''
    
    is_line = lambda s: substr_list(s, line_styles + combo_styles)
    is_marker = lambda s: substr_list(s, marker_styles + combo_styles)
    is_combo = lambda s: substr_list(s, combo_styles)
    
    is_dashed = lambda s: substr_list(s, list(dash_styles.keys()))
    is_bigmarker = lambda s: substr_list(s, big_markers)
    
    reduced_data = reduce(data, stack=True, **reduce_kwargs)
    split_data = panda_unstack(reduced_data)
    
    if reduced_data.shape[1] == 2:
        suffix = ''
    elif reduced_data.shape[1] == 3:
        suffix = '3D'
    else:
        raise NotImplementedError(f'Unsupported number of dimensions; reduced data must be 2d or 3d, given: {reduced_data.shape[1]}')
    
    #TODO: add support for multiple styles (up to one per line)
    #TODO: load in custom plot theme/style from defaults, also allow user to customize
    #TODO: draw bounding box around space
    #TODO: support animation:
    # - camera rotation
    # - window length around current timepoint
    # - opacity of prior timepoints
    # - opacity of current window
    # - opacity of future timepoints    
    #TODO: legend customization
    #TODO: label each point with its original values and/or a user-specified label
    #TODO: support per-datapoint colors, including for lines
    #TODO: support for mixture-based colors-- if 1d, map onto colormap.  If 1 < n-colors <= 3, pick
    #      a color for each dimension and then compute each coordinate as a weighted blend.  If > 3,
    #      map onto 3D colors and then treat as 3D.  Can also define custom functions-- should take
    #      in any observation and return a new vector describing its color (in >= 1D).  also support
    #      a list of functions (one per data matrix)
    #TODO: support plotting of error bars via streamtubes and/or ribbon plots
    #TODO: support plotting "vector fields" by taking a list of two matrices of coordinats,
    #      one specifying the start points and the other specifying the end points of each
    #      vector
    
    #Write a general "hypertools function" that does all analyses by calling normalize --> align --> reduce
    #in sequence.
    
    #TODO: support for nested (>2 level dataframes)
    #TODO: support interpolation for line_styles and combo_styles
    #TODO: define default options outside of this function in a dictionary
    #TODO: normalization options: z-score down rows, across columns, both, or arbitrary functions (take
    #      in a data matrix and return a new matrix of the same type/size).  can also provide lists of
    #      normalization functions to be applied separately to each data matrix.
    #TODO: add support for multiple reduce functions (and/or reduce_kwargs)-- one per data matrix
    #TODO: add hyperalignment and SRM.  also allow different subsets of the data to be aligned, e.g.
    #      according to the unique IDs specified in a list of group IDs, some column of the data matrix,
    #      cluster IDs, or a custom function (or a dictionary of functions specifying how different
    #      group IDs should be aligned); functions should take in a list of data matrices and returned an aligned
    #      list in the same format.
    
    style = style.lower()
    
    assert style in marker_styles + line_styles + combo_styles, f'Unsupported plot style: {style}'
    
    if is_combo(style):
        mode='lines+markers'
    elif is_line(style):
        mode='lines'
    else:
        mode='markers'
    
    if is_line(style):
        if not('line' in kwargs.keys()):
                kwargs['line'] = {}
        #if not('opacity' in kwargs['line'].keys()):
        #    kwargs['line']['opacity'] = float(defaults['plot']['opacity'])
        
        dash = is_dashed(style)
        if dash:
            print(f'dashed line: {dash}')
            if not('dash' in kwargs['line'].keys()):
                kwargs['line']['dash'] = dash_styles[dash]
    if is_marker(style):
        if is_bigmarker(style):
            size = int(defaults['plot']['bigmarkersize'])
        else:
            size = int(defaults['plot']['markersize'])
        
        if not('marker' in kwargs.keys()):
            kwargs['marker'] = {}
        if not('size' in kwargs['marker'].keys()):
            if is_bigmarker(style):
                kwargs['marker']['size'] = int(defaults['plot']['bigmarkersize'])
            else:
                kwargs['marker']['size'] = int(defaults['plot']['markersize'])        
        if not('opacity' in kwargs['marker'].keys()):
            kwargs['marker']['opacity'] = float(defaults['plot']['opacity'])            
    
    names = [str(n) for n in np.array(reduced_data.index.levels[0])]
    
    if reduced_data.shape[1] == 2:
        shapes = [go.Scatter(x=d.iloc[:, 0], y=d.iloc[:, 1], mode=mode, name=names[i], **kwargs) for i, d in enumerate(split_data)]
    elif reduced_data.shape[1] == 3:
        shapes = [go.Scatter3d(x=d.iloc[:, 0], y=d.iloc[:, 1], z=d.iloc[:, 2], mode=mode, name=names[i], **kwargs) for i, d in enumerate(split_data)]
    
    return iplot(shapes)

In [82]:
xs = [np.cumsum(np.random.randn(100, 10), axis=0) for i in np.arange(10)]

In [83]:
x_r = batch_reducer(xs, algorithm='UMAP')

In [85]:
plotter(xs, '.')

In [86]:
plotter(xs, style='-:')

dashed line: -:


In [87]:
plotter(xs, style='-o')

In [88]:
plotter(xs, style='.', reduce_kwargs={'ndims': 2})

In [89]:
plotter(xs, style='-', reduce_kwargs={'ndims': 2})